# Easily export jupyter cells to python module
https://github.com/fastai/course-v3/blob/master/nbs/dl2/notebook2script.py

In [3]:
! python /tf/src/scripts/notebook2script.py evaluation.ipynb

Converted evaluation.ipynb to exp/nb_evaluation.py


In [1]:
%load_ext tensorboard

In [ ]:
%reload_ext tensorboard

In [2]:
cd /tf/src/data/gpt-2/

/tf/src/data/gpt-2


In [3]:
!pip3 install -r requirements.txt

     |████████████████████████████████| 604kB 3.6MB/s eta 0:00:01
     |████████████████████████████████| 61kB 11.1MB/s eta 0:00:01
     |████████████████████████████████| 51kB 15.7MB/s eta 0:00:01
     |████████████████████████████████| 143kB 13.5MB/s eta 0:00:01
     |████████████████████████████████| 122kB 13.4MB/s eta 0:00:01
     |████████████████████████████████| 163kB 13.2MB/s eta 0:00:01
  Stored in directory: /root/.cache/pip/wheels/75/07/38/3c16b529d50cb4e0cd3dbc7b75cece8a09c132692c74450b01
Successfully built regex
  Found existing installation: regex 2019.6.8
    Uninstalling regex-2019.6.8:
      Successfully uninstalled regex-2019.6.8
  Found existing installation: tqdm 4.32.2
    Uninstalling tqdm-4.32.2:
      Successfully uninstalled tqdm-4.32.2
You should consider upgrading via the 'pip install --upgrade pip' command.


In [7]:
import fire
import json
import os
import numpy as np
import tensorflow as tf
import regex as re
from functools import lru_cache
from statistics import median
import argparse
import time
import tqdm
from tensorflow.core.protobuf import rewriter_config_pb2
import glob
import pickle
import h5py

tf.__version__

'2.0.0-beta1'

# Encoding

In [5]:
"""Byte pair encoding utilities"""


@lru_cache()
def bytes_to_unicode():
    """
    Returns list of utf-8 byte and a corresponding list of unicode strings.
    The reversible bpe codes work on unicode strings.
    This means you need a large # of unicode characters in your vocab if you want to avoid UNKs.
    When you're at something like a 10B token dataset you end up needing around 5K for decent coverage.
    This is a signficant percentage of your normal, say, 32K bpe vocab.
    To avoid that, we want lookup tables between utf-8 bytes and unicode strings.
    And avoids mapping to whitespace/control characters the bpe code barfs on.
    """
    bs = list(range(ord("!"), ord("~")+1))+list(range(ord("¡"), ord("¬")+1))+list(range(ord("®"), ord("ÿ")+1))
    cs = bs[:]
    n = 0
    for b in range(2**8):
        if b not in bs:
            bs.append(b)
            cs.append(2**8+n)
            n += 1
    cs = [chr(n) for n in cs]
    return dict(zip(bs, cs))

def get_pairs(word):
    """Return set of symbol pairs in a word.

    Word is represented as tuple of symbols (symbols being variable-length strings).
    """
    pairs = set()
    prev_char = word[0]
    for char in word[1:]:
        pairs.add((prev_char, char))
        prev_char = char
    return pairs

class Encoder:
    def __init__(self, encoder, bpe_merges, errors='replace'):
        self.encoder = encoder
        self.decoder = {v:k for k,v in self.encoder.items()}
        self.errors = errors # how to handle errors in decoding
        self.byte_encoder = bytes_to_unicode()
        self.byte_decoder = {v:k for k, v in self.byte_encoder.items()}
        self.bpe_ranks = dict(zip(bpe_merges, range(len(bpe_merges))))
        self.cache = {}

        # Should haved added re.IGNORECASE so BPE merges can happen for capitalized versions of contractions
        self.pat = re.compile(r"""'s|'t|'re|'ve|'m|'ll|'d| ?\p{L}+| ?\p{N}+| ?[^\s\p{L}\p{N}]+|\s+(?!\S)|\s+""")

    def bpe(self, token):
        if token in self.cache:
            return self.cache[token]
        word = tuple(token)
        pairs = get_pairs(word)

        if not pairs:
            return token

        while True:
            bigram = min(pairs, key = lambda pair: self.bpe_ranks.get(pair, float('inf')))
            if bigram not in self.bpe_ranks:
                break
            first, second = bigram
            new_word = []
            i = 0
            while i < len(word):
                try:
                    j = word.index(first, i)
                    new_word.extend(word[i:j])
                    i = j
                except:
                    new_word.extend(word[i:])
                    break

                if word[i] == first and i < len(word)-1 and word[i+1] == second:
                    new_word.append(first+second)
                    i += 2
                else:
                    new_word.append(word[i])
                    i += 1
            new_word = tuple(new_word)
            word = new_word
            if len(word) == 1:
                break
            else:
                pairs = get_pairs(word)
        word = ' '.join(word)
        self.cache[token] = word
        return word

    def encode(self, text):
        bpe_tokens = []
        for token in re.findall(self.pat, text):
            token = ''.join(self.byte_encoder[b] for b in token.encode('utf-8'))
            bpe_tokens.extend(self.encoder[bpe_token] for bpe_token in self.bpe(token).split(' '))
        return bpe_tokens

    def decode(self, tokens):
        text = ''.join([self.decoder[token] for token in tokens])
        text = bytearray([self.byte_decoder[c] for c in text]).decode('utf-8', errors=self.errors)
        return text

def get_encoder(model_name, models_dir):
    with open(os.path.join(models_dir, model_name, 'encoder.json'), 'r') as f:
        encoder = json.load(f)
    with open(os.path.join(models_dir, model_name, 'vocab.bpe'), 'r', encoding="utf-8") as f:
        bpe_data = f.read()
    bpe_merges = [tuple(merge_str.split()) for merge_str in bpe_data.split('\n')[1:-1]]
    return Encoder(
        encoder=encoder,
        bpe_merges=bpe_merges,
    )

# Model

In [8]:
class HParams():
  n_vocab=50257
  n_ctx=1024
  n_embd=768
  n_head=12
  n_layer=12
  
  def __init__(self, n_vocab, n_ctx, n_embd, n_head, n_layer):
    self.n_vocab = n_vocab
    self.n_ctx = n_ctx
    self.n_embd = n_embd
    self.n_head = n_head
    self.n_layer = n_layer

In [9]:
def default_hparams():
    return HParams(
        n_vocab=50257,
        n_ctx=1024,
        n_embd=768,
        n_head=12,
        n_layer=12,
    )

def shape_list(x):
    """Deal with dynamic shape in tensorflow cleanly."""
    static = x.shape.as_list()
    dynamic = tf.shape(input=x)
    return [dynamic[i] if s is None else s for i, s in enumerate(static)]

def gelu(x):
    return 0.5 * x * (1 + tf.tanh(np.sqrt(2 / np.pi) * (x + 0.044715 * tf.pow(x, 3))))

def norm(x, scope, *, axis=-1, epsilon=1e-5):
    """Normalize to mean = 0, std = 1, then do a diagonal affine transform."""
    with tf.compat.v1.variable_scope(scope):
        n_state = x.shape[-1]
        g = tf.compat.v1.get_variable('g', [n_state], initializer=tf.compat.v1.constant_initializer(1), use_resource=False)
        b = tf.compat.v1.get_variable('b', [n_state], initializer=tf.compat.v1.constant_initializer(0), use_resource=False)
        u = tf.reduce_mean(input_tensor=x, axis=axis, keepdims=True)
        s = tf.reduce_mean(input_tensor=tf.square(x-u), axis=axis, keepdims=True)
        x = (x - u) * tf.math.rsqrt(s + epsilon)
        x = x*g + b
        return x

def split_states(x, n):
    """Reshape the last dimension of x into [n, x.shape[-1]/n]."""
    *start, m = shape_list(x)
    return tf.reshape(x, start + [n, m//n])

def merge_states(x):
    """Smash the last two dimensions of x into a single dimension."""
    *start, a, b = shape_list(x)
    return tf.reshape(x, start + [a*b])

def conv1d(x, scope, nf, *, w_init_stdev=0.02):
    with tf.compat.v1.variable_scope(scope):
        *start, nx = shape_list(x)
        w = tf.compat.v1.get_variable('w', [1, nx, nf], initializer=tf.compat.v1.random_normal_initializer(stddev=w_init_stdev), use_resource=False)
        b = tf.compat.v1.get_variable('b', [nf], initializer=tf.compat.v1.constant_initializer(0), use_resource=False)
        c = tf.reshape(tf.matmul(tf.reshape(x, [-1, nx]), tf.reshape(w, [-1, nf]))+b, start+[nf])
        return c

def attention_mask(nd, ns, *, dtype):
    """1's in the lower triangle, counting from the lower right corner.

    Same as tf.matrix_band_part(tf.ones([nd, ns]), -1, ns-nd), but doesn't produce garbage on TPUs.
    """
    i = tf.range(nd)[:,None]
    j = tf.range(ns)
    m = i >= j - ns + nd
    return tf.cast(m, dtype)


def attn(x, scope, n_state, *, past, hparams):
    assert x.shape.ndims == 3  # Should be [batch, sequence, features]
    assert n_state % hparams.n_head == 0
    if past is not None:
        assert past.shape.ndims == 5  # Should be [batch, 2, heads, sequence, features], where 2 is [k, v]

    def split_heads(x):
        # From [batch, sequence, features] to [batch, heads, sequence, features]
        return tf.transpose(a=split_states(x, hparams.n_head), perm=[0, 2, 1, 3])

    def merge_heads(x):
        # Reverse of split_heads
        return merge_states(tf.transpose(a=x, perm=[0, 2, 1, 3]))

    def mask_attn_weights(w):
        # w has shape [batch, heads, dst_sequence, src_sequence], where information flows from src to dst.
        _, _, nd, ns = shape_list(w)
        b = attention_mask(nd, ns, dtype=w.dtype)
        b = tf.reshape(b, [1, 1, nd, ns])
        w = w*b - tf.cast(1e10, w.dtype)*(1-b)
        return w

    def multihead_attn(q, k, v):
        # q, k, v have shape [batch, heads, sequence, features]
        w = tf.matmul(q, k, transpose_b=True)
        w = w * tf.math.rsqrt(tf.cast(v.shape[-1], w.dtype))

        w = mask_attn_weights(w)
        w = tf.nn.softmax(w, axis=-1)
        a = tf.matmul(w, v)
        return a

    with tf.compat.v1.variable_scope(scope):
        c = conv1d(x, 'c_attn', n_state*3)
        q, k, v = map(split_heads, tf.split(c, 3, axis=2))
        present = tf.stack([k, v], axis=1)
        if past is not None:
            pk, pv = tf.unstack(past, axis=1)
            k = tf.concat([pk, k], axis=-2)
            v = tf.concat([pv, v], axis=-2)
        a = multihead_attn(q, k, v)
        a = merge_heads(a)
        a = conv1d(a, 'c_proj', n_state)
        return a, present


def mlp(x, scope, n_state, *, hparams):
    with tf.compat.v1.variable_scope(scope):
        nx = x.shape[-1]
        h = gelu(conv1d(x, 'c_fc', n_state))
        h2 = conv1d(h, 'c_proj', nx)
        return h2

def block(x, scope, *, past, hparams):
    with tf.compat.v1.variable_scope(scope):
        nx = x.shape[-1]
        a, present = attn(norm(x, 'ln_1'), 'attn', nx, past=past, hparams=hparams)
        x = x + a
        m = mlp(norm(x, 'ln_2'), 'mlp', nx*4, hparams=hparams)
        x = x + m
        return x, present

def past_shape(*, hparams, batch_size=None, sequence=None):
    return [batch_size, hparams.n_layer, 2, hparams.n_head, sequence, hparams.n_embd // hparams.n_head]

def expand_tile(value, size):
    """Add a new axis of given size."""
    value = tf.convert_to_tensor(value=value, name='value')
    ndims = value.shape.ndims
    return tf.tile(tf.expand_dims(value, axis=0), [size] + [1]*ndims)

def positions_for(tokens, past_length):
    batch_size = tf.shape(input=tokens)[0]
    nsteps = tf.shape(input=tokens)[1]
    return expand_tile(past_length + tf.range(nsteps), batch_size)


def model(hparams, X, past=None, scope='model', reuse=tf.compat.v1.AUTO_REUSE):
    with tf.compat.v1.variable_scope(scope, reuse=reuse):
        results = {}
        batch, sequence = shape_list(X)

        wpe = tf.compat.v1.get_variable('wpe', [hparams.n_ctx, hparams.n_embd],
                             initializer=tf.compat.v1.random_normal_initializer(stddev=0.01), use_resource=False)
        wte = tf.compat.v1.get_variable('wte', [hparams.n_vocab, hparams.n_embd],
                             initializer=tf.compat.v1.random_normal_initializer(stddev=0.02), use_resource=False)
        past_length = 0 if past is None else tf.shape(input=past)[-2]
        h = tf.gather(wte, X) + tf.gather(wpe, positions_for(X, past_length))

        # Transformer
        presents = []
        pasts = tf.unstack(past, axis=1) if past is not None else [None] * hparams.n_layer
        assert len(pasts) == hparams.n_layer
        for layer, past in enumerate(pasts):
            h, present = block(h, 'h%d' % layer, past=past, hparams=hparams)
            presents.append(present)
        results['present'] = tf.stack(presents, axis=1)
        h = norm(h, 'ln_f')

        # Language model loss.  Do tokens <n predict token n?
        h_flat = tf.reshape(h, [batch*sequence, hparams.n_embd])
        logits = tf.matmul(h_flat, wte, transpose_b=True)
        logits = tf.reshape(logits, [batch, sequence, hparams.n_vocab])
        results['logits'] = logits
        return results

# Sample from Model

In [25]:
def top_k_logits(logits, k):
    if k == 0:
        # no truncation
        return logits

    def _top_k():
        values, _ = tf.nn.top_k(logits, k=k)
        min_values = values[:, -1, tf.newaxis]
        return tf.compat.v1.where(
            logits < min_values,
            tf.ones_like(logits, dtype=logits.dtype) * -1e10,
            logits,
        )
    return tf.cond(
       pred=tf.equal(k, 0),
       true_fn=lambda: logits,
       false_fn=lambda: _top_k(),
    )


def sample_sequence(*, hparams, length, start_token=None, batch_size=None, context=None, past=None, temperature=1, top_k=0):
    if start_token is None:
        assert context is not None, 'Specify exactly one of start_token and context!'
    else:
        assert context is None, 'Specify exactly one of start_token and context!'
        context = tf.fill([batch_size, 1], start_token)

    def step(hparams, tokens, past=None):
        lm_output = model(hparams=hparams, X=tokens, past=past, reuse=tf.compat.v1.AUTO_REUSE)

        logits = lm_output['logits'][:, :, :hparams.n_vocab]
        presents = lm_output['present']
        presents.set_shape(past_shape(hparams=hparams, batch_size=batch_size))
        return {
            'logits': logits,
            'presents': presents
        }

    def body(past, prev, output, embedding):
        next_outputs = step(hparams, prev, past=past)
        logits = next_outputs['logits'][:, -1, :]  / tf.cast(temperature, dtype=tf.float32)
        logits = top_k_logits(logits, k=top_k)
        samples = tf.random.categorical(logits=logits, num_samples=1, dtype=tf.int32)
        return [
            next_outputs['presents'] if past is None else tf.concat([past, next_outputs['presents']], axis=-2),
            samples,
            tf.concat([output, samples], axis=1),
            logits,
        ]

    past, prev, output, logprobs = body(past, context, context, context)

    def cond(*args):
        return True

#     _, _, tokens, embedding = tf.while_loop(
#         cond=cond, body=body,
#         maximum_iterations=length - 1,
#         loop_vars=[
#             past,
#             prev,
#             output,
#             embedding
#         ],
#         shape_invariants=[
#             tf.TensorShape(past_shape(hparams=hparams, batch_size=batch_size)),
#             tf.TensorShape([batch_size, None]),
#             tf.TensorShape([batch_size, None]),
#             tf.TensorShape([None, None, 768]),
#         ],
#         back_prop=False,
#     )

    probs = tf.math.softmax(logprobs)
    return output, past, probs

In [15]:
from pathlib import Path
def load_dataset(enc, path, combine):
    paths = []
    if os.path.isfile(path):
        # Simple file
        paths.append(path)
    elif os.path.isdir(path):
        # Directory
        for i, (dirpath, _, fnames) in enumerate(os.walk(path)):
            if i % 1000 == 0:
                print(i)
            for fname in fnames:
                paths.append(os.path.join(dirpath, fname))
                
            if i == 10000:
                print("Breaking")
                break
    else:
        # Assume glob
        paths = glob.glob(path)

    token_chunks = []
    raw_text = ''
    for i, path in enumerate(tqdm.tqdm(paths)):
#         if 'after.java' not in path:
#             continue

        try:
            with open(path, 'r') as fp:
                raw_text += fp.read()
#             if len(raw_text) > 35000: continue
            tokens = raw_text# np.stack(enc.encode(raw_text))
            token_chunks.append(tokens)
            raw_text = ''
        except Exception as e:
            print(e)
        if i >= 1000:
            break
    return token_chunks

def binary_search(f, lo, hi):
    if f(lo) or not f(hi):
        return None
    while hi > lo + 1:
        mid = (lo + hi) // 2
        if f(mid):
            hi = mid
        else:
            lo = mid
    return hi


class Sampler(object):
    """Fairly samples a slice from a set of variable sized chunks.

    'Fairly' means that the distribution is the same as sampling from one concatenated chunk,
    but without crossing chunk boundaries."""

    def __init__(self, chunks, seed=None):
        self.chunks = chunks
        self.total_size = sum(chunk.shape[0] for chunk in chunks)
        self.boundaries = [0]
        for i in range(len(chunks)):
            self.boundaries.append(self.boundaries[-1] + chunks[i].shape[0])
        self.rs = np.random.RandomState(seed=seed)

    def sample(self, length):
        assert length < self.total_size // len(
            self.chunks
        ), "Dataset files are too small to sample {} tokens at a time".format(
            length)
        while True:
            index = self.rs.randint(0, self.total_size - length - 1)
            i = binary_search(lambda j: self.boundaries[j] > index, 0,
                              len(self.boundaries) - 1) - 1
            if self.boundaries[i + 1] > index + length:
                within_chunk = index - self.boundaries[i]
                return self.chunks[i][within_chunk:within_chunk + length]

In [16]:
class Args():
    def __init__(self, dataset, model_name, combine, batch_size, learning_rate, optimizer, noise, top_k, top_p, run_name, sample_every, sample_length, sample_num, save_every, val_dataset, val_batch_size, val_batch_count, val_every, pretrained, iterations):
        self.dataset = dataset
        self.model_name = model_name
        self.combine = combine
        self.batch_size = batch_size
        self.learning_rate = learning_rate
        self.optimizer = optimizer
        self.noise = noise
        self.top_k = top_k
        self.top_p = top_p
        self.run_name = run_name
        self.sample_every = sample_every
        self.sample_length = sample_length
        self.sample_num = sample_num
        self.save_every = save_every
        self.val_dataset = val_dataset
        self.val_batch_size = val_batch_size
        self.val_batch_count = val_batch_count
        self.val_every = val_every
        self.pretrained = pretrained
        self.iterations = iterations

In [17]:
args = Args(
                dataset="/tf/src/data/methods/DATA00M_[god-r]/train",
                model_name="117M",
                combine=50000,
                batch_size=1, # DO NOT TOUCH. INCREASING THIS WILL RAIN DOWN HELL FIRE ONTO YOUR COMPUTER.
                learning_rate=0.00002,
                optimizer="sgd",
                noise=0.0,
                top_k=1,
                top_p=0.0,
                run_name="run1",
                sample_every=100,
                sample_length=1023,
                sample_num=1,
                save_every=1000,
                val_dataset=None,
                val_batch_size=1,
                val_batch_count=40,
                val_every=100,
                pretrained=True,
                iterations=200000
    )

In [18]:
enc = get_encoder(args.model_name, "models")
data_set = load_dataset(enc, args.dataset, args.combine)
len(data_set)

0



  0%|          | 0/972771 [00:00<?, ?it/s]

1001

In [ ]:
DATA_SET_SIZE = len(data_set)
TRN_SET_SIZE = int(DATA_SET_SIZE * 0.8)
VAL_SET_SIZE = int(DATA_SET_SIZE * 0.1)
TST_SET_SIZE = int(DATA_SET_SIZE * 0.1)

trn_set = data_set[:TRN_SET_SIZE]
val_set = data_set[TRN_SET_SIZE:TRN_SET_SIZE + VAL_SET_SIZE]
tst_set = data_set[-TST_SET_SIZE:]
len(trn_set), len(val_set), len(tst_set)

In [ ]:
del data_set

In [ ]:
def calc_probs(chkpt_path, ):
    pass

# Self Supervised Pre-Experiment

In [85]:
import math
MAX_CHUNK = 1024

def interact_model(
    model_name='117M',
    seed=None,
    nsamples=1,
    batch_size=1,
    length=None,
    temperature=1,
    top_k=0,
    models_dir='models',
    ds=[]
):
    models_dir = os.path.expanduser(os.path.expandvars(models_dir))
    if batch_size is None:
        batch_size = 1
    assert nsamples % batch_size == 0

    enc = get_encoder("117M", "models")
    hparams = default_hparams()

    if length is None:
        length = hparams.n_ctx // 2
    elif length > hparams.n_ctx:
        raise ValueError("Can't get samples longer than window size: %s" % hparams.n_ctx)

    with tf.compat.v1.Session(graph=tf.Graph()) as sess:
        context = tf.compat.v1.placeholder(tf.int32, [batch_size, None])
        np.random.seed(seed)
        tf.compat.v1.set_random_seed(seed)
        output, past, logits = sample_sequence(
            hparams=hparams, length=length,
            context=context,
            past=None,
            batch_size=batch_size,
            temperature=temperature, top_k=top_k
        )
        

        saver = tf.compat.v1.train.Saver()
        ckpt = tf.train.latest_checkpoint(os.path.join(models_dir, model_name))
        saver.restore(sess, ckpt)
        
        mean_probs = {}
        print(mean_probs)   
        for method in tqdm.tqdm(ds[:2]):
            enc_meth = enc.encode(method)
            rshft = enc_meth[:-1]
            lshft = enc_meth[1:]
            context_tokens = []
            for i, tok in enumerate(rshft):
                context_tokens.append(tok)
                if len(context_tokens) == MAX_CHUNK + 1:
                    context_tokens.pop(0)
               
                out, p, probs = sess.run([output, past, logits], feed_dict={
                    context: [context_tokens for _ in range(batch_size)]
                }, options = tf.compat.v1.RunOptions(report_tensor_allocations_upon_oom = True))
                out = out[:, -1]
                
                print(repr(enc.decode([out[0]])), repr(enc.decode([lshft[i]])), probs[0][out[0]])
                if out[0] == lshft[i]:
                    if enc.decode([out[0]]) in mean_probs:
                        prob, count = mean_probs[enc.decode([out[0]])]
                        mean_probs[enc.decode([out[0]])] = (prob + probs[0][out[0]], count + 1)
                    else:
                        mean_probs[enc.decode([out[0]])] = (probs[0][out[0]], 1)
#                 probs = 
        print(mean_probs.keys())

interact_model(model_name='run1',
    seed=None,
    nsamples=1,
    batch_size=1,
    length=None,
    temperature=1,
    top_k=40,
    models_dir='checkpoint', ds=data_set)




















  0%|          | 0/2 [00:00<?, ?it/s]

{}
'lang' 'java' 0.055041924
'.' '.' 0.9661884
'lang' 'lang' 0.7470425
'.' '.' 0.9985688
'String' 'Override' 0.53475684
').' '\n' 0.012239664
' ' 'public' 0.9716119
' void' ' void' 0.32525918
' on' ' leaf' 0.556629
'Det' 'Set' 0.02382625
'(' 'Entry' 0.55445194
'(' 'Node' 0.78121585
'(' '(' 0.7361973
'java' 'final' 0.84545255
' java' ' java' 0.7972387
'.' '.' 0.99978083
'lang' 'lang' 0.91412145
'.' '.' 0.9995895
'Boo' 'Object' 0.010253676
' cell' ' value' 0.0041451175
')' ')' 0.8313284
' throws' ' throws' 0.25968435
' java' ' java' 0.8929419
'.' '.' 0.9993957
'lang' 'io' 0.9405779
'.' '.' 0.9993592
'Output' 'IO' 0.0079289535
'Exception' 'Exception' 0.99953914
',' ' {' 0.30821943
'\n' '\n' 0.9941788
'public' ' ' 0.2600353
' ' ' ' 0.99484235
' ' ' ' 0.99009025
' android' ' final' 0.0068402616
' java' ' org' 0.86953694
'.' '.' 0.99505275
'spring' 'op' 0.03596117
'itan' 'end' 0.006950536
'em' 'ay' 0.031906605
'.' 'light' 0.8163279
'.' '.' 0.9655592
'com' 'yang' 0.025405843
'a' 'tools' 0.005




















 50%|█████     | 1/2 [00:05<00:05,  5.75s/it]

');' ',' 0.40805316
' value' ' codec' 0.02073386
');' ');' 0.5127268
'\n' '\n' 0.9982705
' ' '}' 0.9998529
'Context' ' void' 0.008007119
' close' ' release' 0.009753885
'();' 'All' 0.19108842
'State' '()' 0.011319834
' throws' ' {' 0.22381838
'\n' '\n' 0.9381627
' ' ' ' 0.9982497
' ' ' ' 0.9985544
' ' ' ' 0.9871023
' ' ' if' 0.87490314
' (' ' ((' 0.529652
'android' 'm' 0.04637747
')' 'Sensor' 0.01310583
'.' 'Manager' 0.4004432
'State' ')' 0.0072070207
' !=' ' !=' 0.552459
' null' ' null' 0.9357989
')' ')' 0.9536101
' {' ' {' 0.93773955
'\n' '\n' 0.9957931
' ' ' ' 0.99968266
' ' ' ' 0.999961
' ' ' ' 0.9999411
' ' ' ' 0.99937195
' ' ' ' 0.98011506
' ' ' ' 0.9574576
' throw' ' ' 0.002757033
' if' ' m' 0.07120687
'Sensor' 'Sensor' 0.89869046
'Manager' 'Manager' 0.9726068
'.' '.' 0.8053988
'release' 'un' 0.3811006
'lock' 'register' 0.49093664
'Image' 'Listener' 0.0030364108
'(' '(' 0.6747008
'm' 'm' 0.5059085
'Activity' 'Sensor' 0.0033035572
'Manager' 'Event' 0.97064304
'.' 'Listener' 0.159




















100%|██████████| 2/2 [00:10<00:00,  5.50s/it]

' ' ' ' 0.99779534
' }' ' }' 0.3917499
'\n' '\n' 0.92069364
' ' '}' 0.99941397
dict_keys(['.', 'lang', ' void', '(', ' java', ')', ' throws', 'Exception', '\n', ' ', 'af', 'ma', ' =', 'Entry', 'Node', ' org', 'op', 'end', 'ay', 'light', 'yang', 'tools', 'ec', '>', 's', 'sche', ');', ' !=', ' null', ' {', 'Sensor', 'Manager', 'm', ' ((', ';', 'Camera', ' m', ' }'])


In [79]:
data_set[1]

'private void releaseAll() {\n    if ((mSensorManager) != null) {\n        mSensorManager.unregisterListener(mSensorEventListener);\n    }\n    if ((mSensorManager) != null)\n        mSensorManager = null;\n    \n    if ((mCamera) != null) {\n        mCamera.setPreviewCallback(null);\n        mCamera.release();\n        mCamera = null;\n    }\n}'

In [65]:
print(mean_probs)

NameError: name 'mean_probs' is not defined

In [72]:
MAX_CHUNK = 3
method = 'stuff'
rshft = method[:-1] # do this on the actual encoding, not plaintext
lshft = method[1:]  # do this on the actual encoding, not plaintext

# enc_meth = enc.encode(method)
context = []
for tok in rshft:
    context.append(tok)
    if len(context) == MAX_CHUNK + 1:
        context.pop(0)
    print(context, len(context))
        # pred, probs <= model(context)
        # if pred equals lshft[something] then update dict with prob for pred tok

['s'] 1
['s', 't'] 2
['s', 't', 'u'] 3
['t', 'u', 'f'] 3


In [42]:
MAX_CHUNK = 2
method = 'stuff'
rshft = method[:-1] # do this on the actual encoding, not plaintext
lshft = method[1:]  # do this on the actual encoding, not plaintext

# enc_meth = enc.encode(method)
for i in range(len(rshft)):
    chunk = rshft[i:MAX_CHUNK + i]
    context = []
    for tok in chunk:
        context.append(tok)
        print(context)
        # pred, probs <= model(context)
        # if pred equals lshft[something] then update dict with prob for pred tok
    print(chunk)
    if MAX_CHUNK + i >= len(rshft): break

['s']
['s', 't']
st
['t']
['t', 'u']
tu
['u']
['u', 'f']
uf


In [ ]:
# API look
mean_probs = calc_probs(file_name)
mean_probs["}"] # => display mean probability of all correctly predicted } characters